# Exam:

## Class: ECON-GA-4004

This exam is meant to be an opportunity to strengthen your understanding of the topics that we have covered in this course.

We hope that in addition to applying what you have learned that you also have some fun.

You are welcome to use books, notes, and any existing materials that you find on the internet, but we ask that:

1. You do not ask any new specific questions on any online forums.
2. You do not collaborate with your classmates (or anyone else).

**Name**: \<INSERT YOUR NAME HERE\>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Asset pricing (40 points)

Please recall the **fundamental  equation of asset pricing theory**:

$$E[ m R^i] = 1 \tag{1}$$

where

\begin{align*}
  E[m R^i] &=1 \quad \text { for } i=1, \ldots, I \\
  m &= \text{ a stochastic discount factor } \\
  R^{i} &= \text { gross return on asset $i$, $i=1, \ldots, I$ } \\
  E &\equiv \text { mathematical expectation }
\end{align*}

and where we assume that $m$ and $R^i, i = 1, \ldots, I$ all have finite variances.

**Question 1** (15 points):

Let $R^f$ be the gross return on a **risk-free** security, meaning that it is a **constant** that is known for sure and uncorrelated with gross returns on all other securities and with the stochastic discount factor.  Please use the fundamental asset pricing equation (1) to derive a formula for $R^f$ in terms the mean and variance of the stochastic discount factor $m$.

**Question 2** (15 points):

Please use the fundamental asset pricing equation (1) to derive the following formula for the expected value of the **excess return** $R^i - R^f$:

$$E R^{i}-R^{f} = -\operatorname{cov}\left(m, R^{i}\right) R^{f} \tag{2} $$

where $\operatorname{cov}\left(m, R^{i}\right)$ is the covariance between $m$ and $R^i$.  

**Question 3** (5 points):

Please use equation (2) to interpret  Lars Peter Hansen's statement  that   "asset pricing theory is all about covariances (and rearranging their implications)". 

**Question 4** (5 points):

Please use equation (2) and properties that you know about variances and covariances to derive the **mean-variance frontier** of asset pricing theory.

## Dynamic programming (45 points)

We consider an infinite time horizon model, i.e. $t = 0, 1, 2, 3\dots$

At $t = 0$ the agent is given a cake with size $\bar{x}$.

The agent must choose how much cake to consume at each time period ($c_t$). The amount of cake that the agent has tomorrow is a random variable that depends on the amount of cake remaining after consumption today.

$$x_{t+1} = \begin{cases} 0.95 (x_t - c_t) \text{ with probability 0.3} \\ 1.02 (x_t - c_t) \text{ with probability 0.7} \end{cases}$$

Consuming quantity $c_t$ of the cake gives current utility $u(c_t)$.

The agent discounts the future with discount factor $\beta$ so that their total utility over the infinite time horizon is

$$v(\bar{x}) = \sum_{t=0}^{\infty} \beta^t u(c_t)$$

$$v(\bar{x}) = u(c_t) + \beta \sum_{t=0}^{\infty} \beta^t u(c_{t+1})$$

We stick with the commonly used CRRA utility function

$$u(c) = \frac{c^{1-\gamma}}{1-\gamma}$$

Use $\bar{x} = 5$, $\gamma = 2$, $\beta = 0.9$ as the parameters of your model

**Question 5** (10 points):

Write the Bellman equation.

\begin{align*}
  v(x_t) &= u(c_t) + \beta E \left[v(x_{t+1}) \right] \\
  v(x_t) &= u(c_t) + \beta \left[ 0.70 \times v(1.02(x_t - c_t)) + 0.30 v(0.95(x_t - c_t))\right] \\
\end{align*}

**Question 6** (20 points):

Numerically compute the policy function and the value function using either value function iteration or policy iteration.

You should use piecewise-linear interpolation to evaluate the functions at points not on the grid.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate as interp
import scipy.optimize as opt

# Define class for convenience of methods and parameter passing
class CakeEating(object):
    def __init__(self, beta=0.9, gamma=2.0, xbar=5.0, nx=75):
        self.beta = beta
        self.gamma = gamma
        self.xbar = xbar
        self.xgrid = np.linspace(1e-2, xbar, nx)

    def u(self, c):
        return c**(1 - self.gamma) / (1 - self.gamma)

    def action_state_value(self, c, x, v):
        """
        Given a value function, `v`, computes
        the value of taking action `c` for
        state `x`
        """
        return self.u(c) + self.beta*(0.70*v(1.02*(x-c)) + 0.30*v(0.95*(x-c)))

In [ ]:
model = CakeEating()

In [ ]:
def bellman_equation(ce, x, v):
    """
    Given a value function `v`, computes the maximum value
    associated with being in state x.
    
    For convenience, it also returns the optimal decision made
    """
    # Optimize 
    sol = opt.minimize_scalar(
        lambda c: -ce.action_state_value(c, x, v),
        bounds=(1e-8, x-1e-10), method="bounded"
    )

    return sol.x, -sol.fun


def value_iteration(ce):
    """
    Computes the optimal policy function and value function
    via value iteration
    """
    v_np1 = np.zeros_like(ce.xgrid)
    v_n = v_np1.copy() + 100.0
    c_star = np.zeros_like(ce.xgrid)

    dist = 100.0
    citer = 0
    while dist > 1e-3:
        citer += 1
        if citer % 25 == 0:
            print(f"Current iteration: {citer}")
            print(f"Current distance: {dist}")

        # Create interpolator
        np.copyto(v_n, v_np1)
        v = interp.interp1d(
            ce.xgrid, v_n,
            fill_value="extrapolate"
        )

        for (i, x) in enumerate(ce.xgrid):
            _c, _v = bellman_equation(ce, x, v)
            c_star[i] = _c
            v_np1[i] = _v
        dist = np.max(np.abs(v_np1 - v_n))
    
    return c_star, v_np1

In [ ]:
interp.interp1d?

In [ ]:
value_iteration(model)

In [ ]:
cstar, vstar = _

**Question 7** (10 points)

Plot the policy function and value function

In [ ]:
plt.plot(vstar)

In [ ]:
plt.plot(cstar)

**Question 8** (5 points)

How much of their initial endowment would the agent be willing to surrender in return for an additional 0.1 consumption in every period (forever)?

_Hint 1_: This is a difficult question. We think it's an interesting question, but, if you are running low on time, then we recommend that you skip it and come back to it.

_Hint 2_: You'll need to compute a new value function where the individual is given 0.1 consumption (in addition to what they choose to consume) in every period.

In [ ]:
model.u(0.1 + 0.05) - model.u(0.05)

In [ ]:
class CakeEatingDeal(objectct):
    def __init__(self, beta=0.9, gamma=2.0, xbar=5.0, nx=75):
        self.beta = beta
        self.gamma = gamma
        self.xbar = xbar
        self.xgrid = np.linspace(1e-2, xbar, nx)

    def u(self, c):
        return c**(1 - self.gamma) / (1 - self.gamma)

    def action_state_value(self, c, x, v):
        """
        Given a value function, `v`, computes
        the value of taking action `c` for
        state `x`
        """
        return self.u(c + 0.1) + self.beta*(0.70*v(1.02*(x-c)) + 0.30*v(0.95*(x-c)))

In [ ]:
model_deal = CakeEatingDeal()

cmod, vmod = value_iteration(model_deal)

In [ ]:
np.searchsorted(model.xgrid, 3.0)

In [ ]:
vstar[45]

In [ ]:
model_deal.xgrid[np.searchsorted(vmod, -118)]

In [ ]:
vstar

In [ ]:
np.searchsorted?

In [ ]:
np.searchsorted(vmod, vstar[-1])

In [ ]:
5 - model_deal.xgrid[9]

## Stationarize the data (40 points)

In this question, we ask you to determine whether a time-series dataset is stationary and, if it isn't, to stationarize it.

Please note that it is okay to just eyeball these time-series datasets and we don't expect you to statistically determine whether they are stationary or not.

They will all take some form that we covered in class (or a combination thereof). For a reminder on what these might look like you can refer to the time-series [notebook](https://github.com/NYU-ComputationalSocialScience/ECON-GA-4004/blob/main/lectures/L06_time_series/02_timeseries.ipynb).

_Hint_: After you stationarize each of the non-stationary time-series datasets, they will have the same stationary component :)


**Dataset 1** (10 points)

In [ ]:
rho = 0.25
sc = np.zeros(150)
for t in range(149):
    sc[t+1] = rho*sc[t] + 0.15*np.random.randn()
np.savetxt("dataset1.csv", sc)

In [ ]:
t = np.arange(150)

In [ ]:
# Load dataset 1
ds1 = np.loadtxt("dataset1.csv")

plt.plot(ds1)

_Is it stationary_?

In [ ]:
# Find the stationary component

**Dataset 2** (10 points)

In [ ]:
lt = 0.05*t + sc
np.savetxt("dataset2.csv", lt)

In [ ]:
plt.plot((ds2 - ds2.mean()) / ds2.std())

In [ ]:
# Load dataset 2
ds2  = np.loadtxt("dataset2.csv")

plt.plot(ds2)

_Is it stationary?_

In [ ]:
# Find the stationary component

**Dataset 3** (10 points)

In [ ]:
f = 1/12

ct = 0.5 + 0.25*np.cos(2*np.pi*f*t) + 0.1*np.sin(2*np.pi*f*t) + sc

np.savetxt("dataset3.csv", ct)

In [ ]:
from sklearn import linear_model

In [ ]:
_f = 1 / 12

x1 = np.cos(2*np.pi*_f*t)
x2 = np.sin(2*np.pi*_f*t)



In [ ]:
lm = linear_model.LinearRegression()

X = np.hstack([x1[:, None], x2[:, None]])
lm.fit(X, ds3)

In [ ]:
plt.plot((ds3 - ds3.mean()) / ds3.std())

In [ ]:
lm.coef_

In [ ]:
plt.plot(ds3 - lm.predict(X))

In [ ]:
import pandas as pd

pd.DataFrame(ds3).diff(12).plot()

In [ ]:
# Load dataset 3
ds3  = np.loadtxt("dataset3.csv")

plt.plot(ds3)

_Is it stationary?_

In [ ]:
# Find the stationary component

**Dataset 4** (10 points)

In [ ]:
slct = lt + ct + sc

np.savetxt("dataset4.csv", slct)

In [ ]:
# Load dataset 4
ds4  = np.loadtxt("dataset4.csv")

plt.plot(ds4)

_Is it stationary?_

In [ ]:
# Find the stationary component